# Projekat - Predviđanje srčane bolesti

### Autor - Nenad Radović, RA18/2020

## Modeli i predikcija

### Učitavanje preprocesiranih podataka

Učitajmo prethodno procesirane podatke.

In [1]:
import pandas as pd
import os

dataset_lr = None
dataset_kt = None

if os.path.exists(path='data_for_lr.csv'):
    dataset_lr = pd.read_csv(filepath_or_buffer='data_for_lr.csv')

if os.path.exists(path='data_for_kt.csv'):
    dataset_kt = pd.read_csv(filepath_or_buffer='data_for_kt.csv')

Potrebno je sada izdvojiti *feature*-e od *target* kolone.

In [2]:
X_lr = dataset_lr.drop(labels=['HeartDisease'], axis='columns')
y_lr = dataset_lr['HeartDisease']

X_kt = dataset_kt.drop(labels=['HeartDisease'], axis='columns')
y_kt = dataset_kt['HeartDisease']

Sada je potrebno napraviti trening i test skupove.

In [3]:
from sklearn.model_selection import train_test_split

X_train_lr, X_test_lr, y_train_lr, y_test_lr = train_test_split(X_lr, y_lr, test_size=0.3, stratify=y_lr)

X_train_kt, X_test_kt, y_train_kt, y_test_kt = train_test_split(X_kt, y_kt, test_size=0.3, stratify=y_kt)

### Modeli

#### Logistička regresija

Prvi model koji ćemo razmotriti jeste model logističke regresije. Učitajmo ga, *fit*-ujmo model i uradimo predikciju na *test* skupu.

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

model_lr = LogisticRegression()
model_knn = KNeighborsClassifier()
model_dt = DecisionTreeClassifier()

Bacimo se na podešavanje i pronalaženje optimalnih hiperparametara za naše modele.

In [5]:
# Hiperparametri logističke regresije
from sklearn.model_selection import GridSearchCV

params_lr = [{
    'C': [i/10 for i in range(1, 13)],
    'tol': [i/100000 for i in range(1, 13)],
    'solver': ['newton-cholesky', 'lbfgs']
}]

gs_lr = GridSearchCV(estimator=model_lr, param_grid=params_lr, cv=9).fit(X=X_train_lr, y=y_train_lr)

Najbolji parametri logističke regresije su

In [6]:
gs_lr.best_params_

{'C': 0.1, 'solver': 'newton-cholesky', 'tol': 1e-05}

Sada smo našli optimalne parametre modela, te ćemo ga istrenirati i predvidjeti *target*-e na test skupu.

In [7]:
model_lr.C = gs_lr.best_params_['C']
model_lr.tol = gs_lr.best_params_['tol']
model_lr.solver = gs_lr.best_params_['solver']

model_lr.fit(X=X_train_lr, y=y_train_lr)
y_pred_lr = model_lr.predict(X=X_test_lr)

Model je sada kreiran i istreniran. Ispišimo neke od karakteristika modela - tačnost, preciznost, odziv, F1 rezultat, kao i matricu konfuzije. Naredne performanse će imati sufiks '*_bf*', što označava su performanse prije smanjivanja dimenzionalnosti modela, odnosno izabiranja najbitnijih *feature* kolona.

In [8]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

as_lr_bf = accuracy_score(y_true=y_test_lr, y_pred=y_pred_lr)
as_lr_bf

0.8392857142857143

In [9]:
ps_lr_bf = precision_score(y_true=y_test_lr, y_pred=y_pred_lr)
ps_lr_bf

0.7933884297520661

In [10]:
rs_lr_bf = recall_score(y_true=y_test_lr, y_pred=y_pred_lr)
rs_lr_bf

0.897196261682243

In [11]:
f1s_lr_bf = f1_score(y_true=y_test_lr, y_pred=y_pred_lr)
f1s_lr_bf

0.8421052631578947

In [12]:
cms_lr_bf = confusion_matrix(y_true=y_test_lr, y_pred=y_pred_lr)
cms_lr_bf

array([[92, 25],
       [11, 96]], dtype=int64)

Uradimo iste postupke na modelu KNN, kao i na stablu odlučivanja.

#### KNN model

In [13]:
# Hiperparametri KNN modela
params_knn = [{
    'n_neighbors': [2*i+1 for i in range(2, 12)],
    'weights': ['uniform', 'distance'],
    'algorithm': ['ball_tree', 'kd_tree', 'brute'],
    'metric': ['euclidean', 'cityblock', 'manhattan']
}]

gs_knn = GridSearchCV(estimator=model_knn, param_grid=params_knn, cv=6).fit(X=X_train_kt, y=y_train_kt)

Najbolji parametri KNN modela su

In [14]:
gs_knn.best_params_

{'algorithm': 'ball_tree',
 'metric': 'cityblock',
 'n_neighbors': 13,
 'weights': 'distance'}

In [15]:
model_knn.n_neighbors = gs_knn.best_params_['n_neighbors']
model_knn.weights = gs_knn.best_params_['weights']
model_knn.algorithm = gs_knn.best_params_['algorithm']
model_knn.metric = gs_knn.best_params_['metric']

model_knn.fit(X=X_train_kt, y=y_train_kt)
y_pred_knn = model_knn.predict(X=X_test_kt)

In [16]:
as_knn_bf = accuracy_score(y_true=y_test_kt, y_pred=y_pred_knn)
as_knn_bf

0.8705357142857143

In [17]:
ps_knn_bf = precision_score(y_true=y_test_kt, y_pred=y_pred_knn)
ps_knn_bf

0.875

In [18]:
rs_knn_bf = recall_score(y_true=y_test_kt, y_pred=y_pred_knn)
rs_knn_bf

0.8504672897196262

In [19]:
f1s_knn_bf = f1_score(y_true=y_test_kt, y_pred=y_pred_knn)
f1s_knn_bf

0.8625592417061612

In [20]:
cms_knn_bf = confusion_matrix(y_true=y_test_kt, y_pred=y_pred_knn)
cms_knn_bf

array([[104,  13],
       [ 16,  91]], dtype=int64)

#### Stablo odluke

In [21]:
# Hiperparametri stabla odluke
params_dt = [{
    'criterion': ['gini', 'entropy', 'log_loss'],
    'splitter': ['best', 'random'],
    'ccp_alpha': [i/10 for i in range(1, 4)],
    'max_depth': [i for i in range(3, 6)]
}]

gs_dt = GridSearchCV(estimator=model_dt, param_grid=params_dt, cv=8).fit(X=X_train_kt, y=y_train_kt)

Najbolji parametri modela stabla odlučivanja su

In [22]:
gs_dt.best_params_

{'ccp_alpha': 0.1, 'criterion': 'gini', 'max_depth': 3, 'splitter': 'best'}

In [23]:
model_dt.criterion = gs_dt.best_params_['criterion']
model_dt.splitter = gs_dt.best_params_['splitter']
model_dt.ccp_alpha = gs_dt.best_params_['ccp_alpha']
model_dt.max_depth = gs_dt.best_params_['max_depth']

model_dt.fit(X=X_train_kt, y=y_train_kt)
y_pred_dt = model_dt.predict(X=X_test_kt)

In [24]:
as_dt_bf = accuracy_score(y_true=y_test_kt, y_pred=y_pred_dt)
as_dt_bf

0.8482142857142857

In [25]:
ps_dt_bf = precision_score(y_true=y_test_kt, y_pred=y_pred_dt)
ps_dt_bf

0.8230088495575221

In [26]:
rs_dt_bf = recall_score(y_true=y_test_kt, y_pred=y_pred_dt)
rs_dt_bf

0.8691588785046729

In [27]:
f1s_dt_bf = f1_score(y_true=y_test_kt, y_pred=y_pred_dt)
f1s_dt_bf

0.8454545454545455

In [28]:
cms_dt_bf = confusion_matrix(y_true=y_test_kt, y_pred=y_pred_dt)
cms_dt_bf

array([[97, 20],
       [14, 93]], dtype=int64)

Sva tri modela daju odlične rezultate pri testiranju, te ne ukazuju na *overfit* nad podacima.

Problem klasifikacije koji rješavamo zahtjeva dobre karakteristike jer je u pitanju ogromna težina problema - nemamo prostora za greške kada se radi o predviđanju toga da li pacijent ima srčanu bolest ili ne. Stoga, potrebno je da naš model pri greškama daje više ***false positive*** vrijednosti no ***false negative*** jer *biće **gora** stvar ako nekoga ko ima srčanu bolest nažalost otpišemo jer naš model predviđa da je nema (**false negative**) nego da uključimo u liječenje i one koji zapravo nemaju i kod kojih će se kasnije zaista ponovnim analizama utvrditi da nemaju (**false positive**).*

Iako želimo da postignemo savršenu "preciznost" (u kontekstu da nemamo ovakvih grešaka pri predviđanjima), naši modeli svakako pokazuju dobru karakteristiku da postoji više *false positive* vrijednosti no *false negative*.

### Smanjenje dimenzionalnosti problema

Iako smo u pretprocesiranju podataka pokušali smanjiti dimenzionalnost problema određivanjem koleracije kako između *feature*-a i *target*-a, tako i između *feature*-a međusobno, sada ćemo pokušati drugom tehnikom da uvidimo da li možemo izbaciti određene *feature*-e ali opet zadržati dobre performanse modela. Tehnika koju ćemo koristiti jeste *SelectKBest*.

Naš model posjeduje 19 *feature*-a trenutno, a mi ćemo za svaki model arbitrarno pokušati smanjiti ih na 10. Kasnije ćemo uporediti dobijene performanse i vidjeti u kojem stepenu smo ih uspjeli održati i nakon izbacivanja nekoliko *feature*-a.

Kreiraćemo nove modele i uporediti njihove performanse sa starim. Naravno, preuzećemo prethodno nađene optimalne hiperparametre starih modela i unijeti ih u nove.

In [29]:
new_model_lr = LogisticRegression()
new_model_knn = KNeighborsClassifier()
new_model_dt = DecisionTreeClassifier()

# Preuzimanje hiperparametara modela logističke regresije
new_model_lr.C = gs_lr.best_params_['C']
new_model_lr.tol = gs_lr.best_params_['tol']
new_model_lr.solver = gs_lr.best_params_['solver']

# Preuzimanje hiperparametara KNN modela
new_model_knn.n_neighbors = gs_knn.best_params_['n_neighbors']
new_model_knn.weights = gs_knn.best_params_['weights']
new_model_knn.algorithm = gs_knn.best_params_['algorithm']
new_model_knn.metric = gs_knn.best_params_['metric']

# Preuzimanje hiperparametara modela stabla odluke
new_model_dt.criterion = gs_dt.best_params_['criterion']
new_model_dt.splitter = gs_dt.best_params_['splitter']
new_model_dt.ccp_alpha = gs_dt.best_params_['ccp_alpha']
new_model_dt.max_depth = gs_dt.best_params_['max_depth']

#### Logistička regresija

In [30]:
from sklearn.feature_selection import SelectKBest, f_classif

k_best_lr = SelectKBest(score_func=f_classif, k=10)
k_best_lr.fit_transform(X=X_train_lr, y=y_train_lr)

new_X_train_lr = X_train_lr.iloc[:, k_best_lr.get_support(indices=True)]
new_X_test_lr = X_test_lr.iloc[:, k_best_lr.get_support(indices=True)]

new_X_train_lr.head()

,Age,FastingBS,MaxHR,Oldpeak,Sex_M,ChestPainType_ATA,ChestPainType_NAP,ExerciseAngina_Y,ST_Slope_Flat,ST_Slope_Up
308,0.461538,0,-0.578947,0.666667,True,False,False,True,True,False
371,0.538462,1,-0.526316,-0.333333,False,True,False,True,False,True
153,-1.000000,0,0.526316,-0.333333,True,True,False,False,False,True
130,-1.230769,0,-0.263158,-0.333333,True,False,True,False,False,True
609,-0.076923,1,0.315789,0.466667,True,False,True,False,False,False


In [31]:
new_model_lr.fit(X=new_X_train_lr, y=y_train_lr)
new_y_pred_lr = new_model_lr.predict(X=new_X_test_lr)

Uporedimo performanse starog modela sa više *feature*-a i novog modela sa manje pomenutih tako što ćemo oduzeti prethodne performanse od trenutnih. Ako dobijemo negativnu razliku, znači da je model poboljšao performanse - u suprotnom, "gore" su.

In [32]:
as_lr_bf - accuracy_score(y_true=y_test_lr, y_pred=new_y_pred_lr)

0.0

In [33]:
ps_lr_bf - precision_score(y_true=y_test_lr, y_pred=new_y_pred_lr)

-0.00493089797902635

In [34]:
rs_lr_bf - recall_score(y_true=y_test_lr, y_pred=new_y_pred_lr)

0.009345794392523366

In [35]:
f1s_lr_bf - f1_score(y_true=y_test_lr, y_pred=new_y_pred_lr)

0.0013972985561248041

In [36]:
cms_lr_bf - confusion_matrix(y_true=y_test_lr, y_pred=new_y_pred_lr)

array([[-1,  1],
       [-1,  1]], dtype=int64)

Očigledno je da su razlike performansi gotovo nepostojeće te ne gubimo validnost modela pri smanjenju dimenzionalnosti. Provjerimo kakav je slučaj kod KNN modela, kao i stabla odlučivanja.

#### KNN model i stablo odlučivanja

In [37]:
k_best_kt = SelectKBest(score_func=f_classif, k=10)
k_best_kt.fit_transform(X=X_train_kt, y=y_train_kt)

new_X_train_kt = X_train_kt.iloc[:, k_best_kt.get_support(indices=True)]
new_X_test_kt = X_test_kt.iloc[:, k_best_kt.get_support(indices=True)]

new_X_train_kt.head()

,MaxHR,Oldpeak,Sex_F,Sex_M,ChestPainType_ASY,ChestPainType_ATA,ExerciseAngina_N,ExerciseAngina_Y,ST_Slope_Flat,ST_Slope_Up
496,0.921053,0.066667,True,False,False,True,True,False,True,False
4,-0.473684,-0.333333,False,True,False,False,True,False,False,True
538,0.289474,-0.066667,False,True,True,False,True,False,True,False
303,0.000000,0.000000,False,True,True,False,False,True,True,False
256,0.842105,-0.333333,False,True,False,False,True,False,False,True


In [38]:
new_model_knn.fit(X=new_X_train_kt, y=y_train_kt)
new_model_dt.fit(X=new_X_train_kt, y=y_train_kt)

new_y_pred_knn = new_model_knn.predict(X=new_X_test_kt)
new_y_pred_dt = new_model_dt.predict(X=new_X_test_kt)

KNN model

In [39]:
as_knn_bf - accuracy_score(y_true=y_test_kt, y_pred=new_y_pred_knn)

0.008928571428571397

In [40]:
ps_knn_bf - precision_score(y_true=y_test_kt, y_pred=new_y_pred_knn)

0.01650943396226412

In [41]:
rs_knn_bf - recall_score(y_true=y_test_kt, y_pred=new_y_pred_knn)

0.0

In [42]:
f1s_knn_bf - f1_score(y_true=y_test_kt, y_pred=new_y_pred_knn)

0.008099147809447538

In [43]:
cms_knn_bf - confusion_matrix(y_true=y_test_kt, y_pred=new_y_pred_knn)

array([[ 2, -2],
       [ 0,  0]], dtype=int64)

Model stabla odluke

In [44]:
as_dt_bf - accuracy_score(y_true=y_test_kt, y_pred=new_y_pred_dt)

0.0

In [45]:
ps_dt_bf - precision_score(y_true=y_test_kt, y_pred=new_y_pred_dt)

0.0

In [46]:
rs_dt_bf - recall_score(y_true=y_test_kt, y_pred=new_y_pred_dt)

0.0

In [47]:
f1s_dt_bf - f1_score(y_true=y_test_kt, y_pred=new_y_pred_dt)

0.0

In [48]:
cms_dt_bf - confusion_matrix(y_true=y_test_kt, y_pred=new_y_pred_dt)

array([[0, 0],
       [0, 0]], dtype=int64)

Za svaki model dobijamo da su maksimalna odstupanja u rangu od &plusmn;3%, što znači da smo uspjeli smanjiti dimenzionalnost problema bez značajnog gubitka na performansama. Korisniku ostavljamo da pokuša da još više degradira dimenzionalnost te uporedi performanse.

### Finalni zaključak

Modeli koji se najuspješnije pokazuju kroz mnogostruka testiranja jesu model logističke regresije, kao i KNN model. Generalno se najlošije pokazuje model stabla odluke, ali ne za ogroman gubitak performansi. Zaključak jeste da modeli mogu vrlo dobro (do 90% preciznosti, *napomena da preciznost nije uvijek dobar indikator dobrog modela*) opisati sistem i predvidjeti da li neko ima srčanu bolest, uz mali procenat pogrešnih procjena. 